In [8]:
import pandas as pd
from collections import Counter

df_niu = pd.read_csv("Snow_Tagging/SnowTagging_Niu_ROUND4_1_tagged.csv")
df_cat = pd.read_csv("Snow_Tagging/SnowTagging_Cat_ROUND4_1_tagged.csv")
df_kathy = pd.read_csv("Snow_Tagging/SnowTagging_Kathy_ROUND4_1_tagged.csv")
assert(len(df_niu) == len(df_cat) == len(df_kathy))


In [9]:
a = [[1,2],2,3]
b = [2,3,5]
aa = ["a","b"]
new_df = pd.DataFrame([a,b]).transpose()
new_df.columns = aa
type(new_df.a[0])

list

In [3]:
KEYWORD = {"match_all": "all same",
           "match_two": "two same",
           "match_none": "none same"}
QUESTIONS = {"diagnostic":"Q1",
             "mention":"Q2_1",
             "prognostic":"Q2_2",
             "motivational_action":"Q3_1",
             "motivational_rationale":"Q3_2"}

def toList(row):
    taglist = row[1:-1].split(";")
    taglist = ["video_not_included" if tag=="" else tag for tag in taglist]
    return taglist

def generateResultForRow(question_list, row_cat, row_niu, row_kathy):
    result = {
        "embedUrl":row_cat["embedUrl"],
        "videoId":row_cat["videoId"],
        "title":row_cat["title"]}
        
    for question in question_list:
        question_cat = set(sorted(toList(row_cat[question])))
        question_niu = set(sorted(toList(row_niu[question])))
        question_kathy = set(sorted(toList(row_kathy[question])))

        question_match = question_cat & question_niu & question_kathy
        question_match_two =    {"CatandNiu":(question_cat & question_niu) or "no_common",
                                "CatandKathy":(question_cat & question_kathy) or "no_common",
                                "KathyandNiu":(question_niu & question_kathy) or "no_common"}
        for i in ["CatandNiu","CatandKathy","KathyandNiu"]:
            if question_match_two[i] == "no_common":
                del question_match_two[i]

        question_match_all = False
        if len(question_match) == 0:
            question_match = {}
        if len(question_match) == len(question_cat) == len(question_niu) == len(question_kathy):
            question_match_all = True

        sameness = ""
        if question_match_all:
            sameness = KEYWORD["match_all"]
        elif len(question_match_two)>0:
            sameness = KEYWORD["match_two"]
        else:
            sameness = KEYWORD["match_none"]

        result[f"{QUESTIONS[question]}_Cat_tag"] = ','.join(question_cat)
        result[f"{QUESTIONS[question]}_Kathy_tag"] = ','.join(question_kathy)
        result[f"{QUESTIONS[question]}_Niu_tag"] = ','.join(question_niu)
        result[f"{QUESTIONS[question]}_same"] = sameness
        result[f"{QUESTIONS[question]}_two_same"] = ', '.join('{} : {}'.format(key, ','.join(value)) for key, value in question_match_two.items()) if len(question_match_two)>0 else ""
        
    return result

results = []
for index, row_cat in df_cat.iterrows():
    row_niu = df_niu.iloc[index]
    row_kathy = df_kathy.iloc[index]
    result = generateResultForRow(QUESTIONS, row_cat, row_niu, row_kathy)
    results.append(result)
        
# # cols = results[0].keys()
df_results = pd.DataFrame(results)
df_results.to_csv(f"Snow_Tagging/ROUND4_1_agree_rate_TESTING.csv", index=False)

samelist = []
samelist.append([(f"{q}",item[f"{q}_same"]) for item in results for q in QUESTIONS.values()])
[sorted(Counter(q).items(), key= lambda i:i[0]) for q in samelist]


[[(('Q1', 'all same'), 35),
  (('Q1', 'none same'), 2),
  (('Q1', 'two same'), 13),
  (('Q2_1', 'all same'), 30),
  (('Q2_1', 'none same'), 2),
  (('Q2_1', 'two same'), 18),
  (('Q2_2', 'all same'), 34),
  (('Q2_2', 'none same'), 1),
  (('Q2_2', 'two same'), 15),
  (('Q3_1', 'all same'), 25),
  (('Q3_1', 'none same'), 2),
  (('Q3_1', 'two same'), 23),
  (('Q3_2', 'all same'), 26),
  (('Q3_2', 'none same'), 2),
  (('Q3_2', 'two same'), 22)]]

In [7]:
dfcat_final = pd.read_csv("Snow_Tagging/SnowTagging_Cat_FINALROUND.csv")
list(dfcat_final["prognostic"])

['[spread_the_word]',
 '[share_videos]',
 '[gameplay]',
 '[spread_the_word]',
 '[]',
 '[creation]',
 '[criticism]',
 '[gameplay]',
 '[spread_the_word]',
 '[spread_the_word]',
 '[creation]',
 '[spread_the_word]',
 '[spread_the_word]',
 '[environmental_discussion]',
 '[gameplay]',
 '[spread_the_word]',
 '[creation]',
 '[knowledge]',
 '[spread_the_word]',
 '[gameplay]',
 '[knowledge]',
 '[knowledge]',
 '[creation]',
 '[spread_the_word]',
 '[spread_the_word]',
 '[gameplay]',
 '[spread_the_word]',
 '[spread_the_word]',
 '[gameplay]',
 '[gameplay]',
 '[plant_trees]',
 '[knowledge]',
 '[knowledge]',
 '[gameplay]',
 '[spread_the_word]',
 '[gameplay]',
 '[none_apply]',
 '[spread_the_word]',
 '[gameplay]',
 '[creation]',
 '[spread_the_word;plant_trees]',
 '[creation]',
 '[]',
 '[donation]',
 '[creation]',
 '[creation]',
 '[spread_the_word]',
 '[spread_the_word]',
 '[plant_trees]',
 '[gameplay]',
 '[spread_the_word]',
 '[gameplay]',
 '[share_videos]',
 '[share_videos;gameplay]',
 '[creation]',
 '